In [ ]:
# !pip install faiss-cpu

# Library

In [4]:
import os


os.chdir("../")

In [5]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter

from langchain.embeddings import HuggingFaceEmbeddings

from langchain_experimental.text_splitter import SemanticChunker
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Load Emberding Model

In [6]:
device = 'cpu'
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',
                                      model_kwargs = {'device': device})

C:\Users\User\AppData\Local\Temp\ipykernel_612\3614191810.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',


# Load Vector DB

In [7]:

# Load the Vector Database
vectorstore = FAISS.load_local(folder_path="vector_db/faiss_index_hybride",
                               embeddings=embedding_model,
                               allow_dangerous_deserialization=True)

# Query

In [9]:
# query_text = "What clues did Aragorn find on the hill that helped him track Frodo, and how did his observations influence his next actions?"
query_text = "what is  chapters of the two towers?"
emd_query = embedding_model.embed_query(query_text)

results_emd = vectorstore.similarity_search_with_score_by_vector(emd_query, k=100)

#  when vector rize contentx its should be unique\
results_emd[0][0].page_content
for result in results_emd:
    print(f"{result[0].page_content} -> {result[1]}")
    # print("Metadata:", result.metadata)
    print("------")

THE TWO
TOWERS
being the second part
of
The Lord of the Rings -> 0.6816160082817078
------
iii
THE TWO
TOWERS
being the second part of
THE LORD OF THE RINGS
by
J.R.R. TOLKIEN -> 0.7469179034233093
------
This second part, The Two Towers, now tells how each of
the members of the Fellowship of the Ring fared, after the
breaking of their fellowship, until the coming of the great
Darkness and the outbreak of the War of the Ring, which is
to be recounted in the third and last part. -> 0.8021478652954102
------
714 the two towers
helped in their making long ago? -> 0.8128993511199951
------
The second part, (Books Three and Four), The Two
Towers, recounted the deeds of all the Company after the
breaking of the Fellowship of the Ring. -> 0.8134757280349731
------
926 the two towers
across in time? -> 0.8608662486076355
------
750 the two towers
‘We understand it all perfectly now,’ said Gimli. ‘All except one thing,’ said Aragorn: ‘leaf from the South-
farthing in Isengard. -> 0.8927507400512

# Hybide Query

In [1]:
from whoosh.index import open_dir
from whoosh.qparser import QueryParser

In [2]:
KEY_WORD_BASE_INDEX_DIR="key_word_index"

In [8]:
ix = open_dir(KEY_WORD_BASE_INDEX_DIR)

In [71]:

def bm25_retrieve(query_text, index, limit=50):
    with index.searcher() as searcher:
        parser = QueryParser("content", index.schema)
        query = parser.parse(query_text)
        bm25_results = searcher.search(query, limit=limit)
        # Extract the unique document IDs from BM25 results.
        candidate_ids = set(hit["page"] for hit in bm25_results)
        results=[(hit.fields(),hit.score) for hit in bm25_results]
    return candidate_ids, results


In [34]:
query_text = "book three contents"

In [50]:
emd_query = embedding_model.embed_query(query_text)

results_emd = vectorstore.similarity_search_with_score_by_vector(emd_query, k=1000)

In [72]:

# Example usage:
candidate_ids, bm25_hits = bm25_retrieve(query_text, ix, limit=50)

## Keyword search

In [60]:
candidate_ids

{5, 9}

In [73]:
for hit in bm25_hits:
    print(hit[0]['content'])
    print(hit[1])
    print("------")

CONTENTS
Note on the Text                                                     ix
Note on the 50th Anniversary Edition                  xviii
Foreword to the Second Edition                       xxiii
Prologue Concerning Hobbits, and other
matters 1
book one
I A Long-expected Party 27
II The Shadow of the Past 55
III Three is Company 85
IV A Short Cut to Mushrooms 112
V A Conspiracy Unmasked 128
VI The Old Forest 143
VII In the House of Tom Bombadil 161
VIII Fog on the Barrow-downs 176
IX At the Sign of The Prancing Pony 195
X Strider 213
XI A Knife in the Dark 230
XII Flight to the Ford 257
book two
I Many Meetings 285
II The Council of Elrond 311
III The Ring Goes South 354
19.155808363908
------
CONTENTS
Synopsis vii
book three
I The Departure of Boromir 537
II The Riders of Rohan 547
III The Uruk-hai 578
IV Treebeard 600
V The White Rider 636
VI The King of the Golden Hall 660
VII Helm’s Deep 686
VIII The Road to Isengard 708
IX Flotsam and Jetsam 730
X The Voice of Saruman 751
XI T

## Vector results

In [52]:
for result in results_emd:
    print(f"{result[0].page_content} -> {result[1]}")
    print("Page:", result[0].metadata['page'])
    print("------")

BOOK THREE -> 0.3967073857784271
Page: 13
------
BOOK TWO -> 0.6825447082519531
Page: 311
------
BOOK FOUR -> 0.7516127824783325
Page: 263
------
BOOK SIX -> 0.8016297817230225
Page: 212
------
BOOK FIVE -> 0.8371295928955078
Page: 16
------
BOOK ONE -> 0.8458421230316162
Page: 53
------
Meanwhile Tolkien worked to keep a promise
he had made in the foreword to volume one: that ‘an index
of names and strange words’ would appear in the third vol-
ume. -> 0.9667881727218628
Page: 9
------
xii note on the text
cause of the delay in publishing volume three, which in the
end contained no index at all, only an apology from the
publisher for its absence. -> 1.018458604812622
Page: 10
------
These chapters, eventually to become Book Four,
were written and sent out as a serial to my son, Christopher,
then in South Africa with the RAF. -> 1.0315533876419067
Page: 24
------
iii
THE TWO
TOWERS
being the second part of
THE LORD OF THE RINGS
by
J.R.R. TOLKIEN -> 1.0319812297821045
Page: 3
------
I ha

## filter or Re rank

In [53]:
# Filter FAISS results based on candidate_ids from BM25 search.
hybrid_results = [(doc,score) for doc, score in results_emd if doc.metadata.get("page") in candidate_ids]


In [54]:

# Optionally, you can re-rank these hybrid_results if needed (for example, by combining the BM25 rank and the FAISS score)
for doc in hybrid_results:
    print(f"{doc[0].page_content} -> {doc[1]}")
    print("------")

Meanwhile Tolkien worked to keep a promise
he had made in the foreword to volume one: that ‘an index
of names and strange words’ would appear in the third vol-
ume. -> 0.9667881727218628
------
Tolkien’s The Lord of the Rings is often erroneously
called a trilogy, when it is in fact a single novel, consisting
of six books plus appendices, sometimes published in three
volumes. -> 1.0594117641448975
------
Book Three told of
the repentance and death of Boromir, and of his funeral in
a boat committed to the Falls of Rauros; of the capture
of Meriadoc and Peregrin by orc-solders, who bore them
towards Isengard over the eastern plains of Rohan; and of
their pursuit by Aragorn, Legolas and Gimli. -> 1.071060061454773
------
The second part, (Books Three and Four), The Two
Towers, recounted the deeds of all the Company after the
breaking of the Fellowship of the Ring. -> 1.093337059020996
------
The ﬁrst volume, The Fellowship of the Ring, was published
in Great Britain by the London ﬁrm Geor